# 01 - Exploratory Data Analysis
## Awesome Foundation Signal Score

Corpus-level statistics, chapter distributions, label rates, and text field analysis
across 64,000+ grant applications.

**Data source:** Parameterized via `AWESOMEBITS_DB` env var (defaults to `.scratch/data/awesomebits.duckdb`).

In [ ]:
import sys
sys.path.insert(0, '.')
from helpers import *
import pandas as pd
import numpy as np

setup_plotting()
con = connect()
print(f'Connected to: {DB_PATH}')
for t in con.execute('SHOW TABLES').fetchall():
    cnt = con.execute(f'SELECT count(*) FROM {t[0]}').fetchone()[0]
    print(f'  {t[0]}: {cnt:,} rows')

## Corpus Overview

In [ ]:
df = con.execute('SELECT * FROM projects').df()
print(f'Total applications: {len(df):,}')
print(f'Date range: {df.created_at.min()} to {df.created_at.max()}')
print(f'Funded: {df.funded_on.notna().sum():,} ({df.funded_on.notna().mean():.1%})')
print(f'Hidden: {df.hidden_at.notna().sum():,} ({df.hidden_at.notna().mean():.1%})')
print(f'Unlabeled: {(df.funded_on.isna() & df.hidden_at.isna()).sum():,}')

## Chapter Distribution

In [ ]:
chapters = con.execute('''
    SELECT c.name, c.country, c.inactive_at IS NOT NULL as inactive,
        COUNT(*) as total,
        COUNT(p.funded_on) as funded,
        COUNT(p.hidden_at) as hidden,
        COUNT(*) - COUNT(p.funded_on) - COUNT(p.hidden_at) as unlabeled
    FROM projects p
    JOIN chapters c ON p.chapter_id = c.id
    GROUP BY c.name, c.country, inactive
    ORDER BY total DESC
''').df()

print(f'Active chapters: {(~chapters.inactive).sum()}')
print(f'Inactive chapters: {chapters.inactive.sum()}')
print(f'Countries: {chapters.country.nunique()}')
print()
print('Top 15 chapters by volume:')
print(chapters.head(15)[['name', 'country', 'total', 'funded', 'hidden']].to_string(index=False))

In [ ]:
# Applications per chapter (top 20)
top20 = chapters.head(20).copy()
fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(top20['name'], top20['total'], color='steelblue')
ax.set_xlabel('Applications')
ax.set_title('Top 20 Chapters by Application Volume')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## Label Rates by Chapter

Which chapters actively use the hide feature? This determines which chapters have enough ground truth for model validation.

In [ ]:
# Chapters with enough labeled data for validation
validatable = chapters[(chapters.funded >= 10) & (chapters.hidden >= 10)].copy()
validatable['label_rate'] = ((validatable.funded + validatable.hidden) / validatable.total * 100).round(1)
validatable['fund_rate'] = (validatable.funded / (validatable.funded + validatable.hidden) * 100).round(1)

print(f'Chapters with >= 10 funded AND >= 10 hidden: {len(validatable)}')
print()
print(validatable[['name', 'country', 'total', 'funded', 'hidden', 'label_rate', 'fund_rate']].to_string(index=False))

## Temporal Distribution

In [ ]:
df['year'] = pd.to_datetime(df.created_at).dt.year
yearly = df.groupby('year').agg(
    total=('id', 'count'),
    funded=('funded_on', lambda x: x.notna().sum()),
    hidden=('hidden_at', lambda x: x.notna().sum()),
).reset_index()
yearly['unlabeled'] = yearly.total - yearly.funded - yearly.hidden

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(yearly.year, yearly.unlabeled, label='Unlabeled', color='lightgray')
ax.bar(yearly.year, yearly.hidden, bottom=yearly.unlabeled, label='Hidden', color='salmon')
ax.bar(yearly.year, yearly.funded, bottom=yearly.unlabeled + yearly.hidden, label='Funded', color='mediumseagreen')
ax.set_xlabel('Year')
ax.set_ylabel('Applications')
ax.set_title('Applications by Year and Label')
ax.legend()
plt.tight_layout()
plt.show()

## Text Field Analysis

In [ ]:
# Word counts per field
for field in TEXT_FIELDS:
    df[f'wc_{field}'] = df[field].fillna('').apply(word_count)
df['wc_total'] = df['wc_about_me'] + df['wc_about_project'] + df['wc_use_for_money']

print('Word count statistics:')
print(df[['wc_about_me', 'wc_about_project', 'wc_use_for_money', 'wc_total']].describe().round(1))

In [ ]:
# Word count distributions by label
df['label'] = df.apply(label_project, axis=1)

fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
for ax, field in zip(axes, TEXT_FIELDS):
    for label, color in [('funded', 'mediumseagreen'), ('hidden', 'salmon')]:
        subset = df[df.label == label][f'wc_{field}']
        ax.hist(subset, bins=50, alpha=0.6, color=color, label=label, range=(0, 1000))
    ax.set_title(field.replace('_', ' ').title())
    ax.set_xlabel('Word Count')
    ax.legend()
axes[0].set_ylabel('Frequency')
plt.suptitle('Word Count Distribution: Funded vs Hidden', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Empty field rates by label
for label in ['funded', 'hidden', 'unlabeled']:
    subset = df[df.label == label]
    empties = {f: (subset[f].fillna('').str.strip() == '').mean() for f in TEXT_FIELDS}
    print(f'{label:>10}: ' + '  '.join(f'{f}={v:.1%}' for f, v in empties.items()))

## Geographic Distribution

In [ ]:
country_stats = con.execute('''
    SELECT c.country, COUNT(*) as apps, COUNT(DISTINCT c.id) as chapters
    FROM projects p JOIN chapters c ON p.chapter_id = c.id
    GROUP BY c.country ORDER BY apps DESC
''').df()

fig, ax = plt.subplots(figsize=(12, 6))
top_countries = country_stats.head(15)
ax.barh(top_countries.country, top_countries.apps, color='steelblue')
ax.set_xlabel('Applications')
ax.set_title('Applications by Country (Top 15)')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print(f'\nTotal countries: {len(country_stats)}')
print(f'English-speaking: {country_stats[country_stats.country.isin(ENGLISH_SPEAKING_COUNTRIES)].apps.sum():,} apps')